In [1]:
#Import libraries
import pandas as pd
from datetime import datetime
import time

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import nltk
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

from wordcloud import WordCloud, STOPWORDS

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import time
import string

from sklearn.feature_extraction import text
pd.set_option('display.max_columns', 500)
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# Let's call all the functions
pd.set_option('display.max_colwidth', 100)
nltk.download('wordnet')

# NLTK(Natural Language Toolkit) in python has a list of stopwords stored in 16 different languages.
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ANNITAN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
#importing datasets
python = pd.read_csv('python.csv')
SQL = pd.read_csv('SQL.csv')

#Change columns names to lowercase
python.columns = python.columns.str.lower().str.replace(' ', '_')
SQL.columns = SQL.columns.str.lower().str.replace(' ', '_')

C:\Users\ANNITAN\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (75,79,80,82) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\ANNITAN\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (81,82) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
#Only select the 2 Columns to analyze
python = python[['selftext', 'title']]
SQL = SQL[['selftext', 'title']]

In [5]:
#Check first few rows of python
python.head(2)

,selftext,title
0,NaN,Know what is web scraping with python
1,NaN,How to Learn Python for Data Science? The Best Ways


In [6]:
#Check first few rows of R
SQL.head(2)

,selftext,title
0,Database: Redshift\n\nHi everyone! I have a question. I'm trying to figure out how long a user v...,Find the next timestamp based on where condition per user
1,1) A car salesman at Capital Motors gets a tip that other salesmen are selling cars for way chea...,SQL help please


In [7]:
#Check shape
python.shape, SQL.shape

((50860, 2), (13149, 2))

In [8]:
#Understand the dataset
python.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50860 entries, 0 to 50859
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   selftext  35994 non-null  object
 1   title     50860 non-null  object
dtypes: object(2)
memory usage: 794.8+ KB


In [9]:
#Understand the dataset
SQL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13149 entries, 0 to 13148
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   selftext  11368 non-null  object
 1   title     13148 non-null  object
dtypes: object(2)
memory usage: 205.6+ KB


In [10]:
#Check for null
python.isna().sum()

selftext    14866
title           0
dtype: int64

In [11]:
#Check for null
SQL.isna().sum()

selftext    1781
title          1
dtype: int64

In [12]:
#Since we have enough sample size, let's drop those rows with null values.
# dropna first, if not function will throw error as null values are considered floats and cannot .lower()
python.dropna(subset=['selftext'], inplace=True)
SQL.dropna(subset=['selftext'], inplace=True)

In [13]:
#Check for null
python.isna().sum()

selftext    0
title       0
dtype: int64

In [14]:
#Check for null
SQL.isna().sum()

selftext    0
title       1
dtype: int64

In [15]:
#Check the number of rows with 'selftext' = [removed]
python.loc[python['selftext'] == '[removed]']

,selftext,title
5,[removed],How to add tooltip feature using mplfinance library in python?
7,[removed],Are python learning books worth it ? (If so any recommendations? Pls:'))
19,[removed],Python Webscraping Cookies
21,[removed],SQL gets returned with decimals on int value
22,[removed],IF CONDITIONS...less than/greater than...not returning print statement?
...,...,...
50849,[removed],Python without pip
50850,[removed],Intro to Python Tutorial?
50856,[removed],"reinstalling opencv-python ""broke"" the package for me, what do I do?"
50857,[removed],Trying to Create an app that displays live game data


In [16]:
#Check the number of rows with 'selftext' = [removed]
SQL.loc[SQL['selftext'] == '[removed]']

,selftext,title
103,[removed],Google API turn Address into Lat &amp; Long
150,[removed],SQL server doesnt want to go out of my computer
168,[removed],LISTAGG returning duplicate results using oracledb 12
367,[removed],What is the Future of SQL in Data Science?
442,[removed],SQL query to delete duplicates in 2 columns
...,...,...
12648,[removed],Looking for feedback - SQL Debugger Project
12654,[removed],MYSQL vs MONGODB
12796,[removed],[MYSQL] [HeidiSQL] an SQL query that changes all values greater than 1 in a specific column into...
12885,[removed],How would you update this database


In [17]:
#Drop the rows with 'selftext' = [removed]
python.drop(python.loc[python['selftext'] == '[removed]'].index, inplace=True)
SQL.drop(SQL.loc[SQL['selftext'] == '[removed]'].index, inplace=True)

In [18]:
#Check shape
python.shape, SQL.shape

((18200, 2), (11217, 2))

In [19]:
#Drop the rows with 'selftext' = [deleted]
python.drop(python.loc[python['selftext'] == '[deleted]'].index, inplace=True)
SQL.drop(SQL.loc[SQL['selftext'] == '[deleted]'].index, inplace=True)

In [20]:
#Check shape
python.shape, SQL.shape

((17868, 2), (11132, 2))

In [21]:
#Check Dataset
python.head()

,selftext,title
3,"Hey Python-Friends,\n\nIn the last couple of weeks and months I used these tough times and lockd...",SolarY - A Solar System / Space Science Python Library
6,"a = 5 \nb = a \na = 11 \nMy question, as a beginner in python, is what is the value of b aft...",Question on assignments in Python3: What is the value of b after changing the value of a ?
8,Can anyone give me any direction on how to code in Python a script that copies 2 or more columns...,CSV - Copy Certain Columns From One CV to Another
9,"Today, I just decided to make some Tensorflow stuffs in Python. (I don't use Python for \~3 year...",This is the most outrageous command output I have ever seen in my entire Unix life
11,Hi guys!\n\nI have found that most of the packages and explanation of FP Growth algorithm in dat...,Python implementation of FP Growth algorithm in data mining


In [22]:
#Reset Index
python = python.reset_index()
python.head()

,index,selftext,title
0,3,"Hey Python-Friends,\n\nIn the last couple of weeks and months I used these tough times and lockd...",SolarY - A Solar System / Space Science Python Library
1,6,"a = 5 \nb = a \na = 11 \nMy question, as a beginner in python, is what is the value of b aft...",Question on assignments in Python3: What is the value of b after changing the value of a ?
2,8,Can anyone give me any direction on how to code in Python a script that copies 2 or more columns...,CSV - Copy Certain Columns From One CV to Another
3,9,"Today, I just decided to make some Tensorflow stuffs in Python. (I don't use Python for \~3 year...",This is the most outrageous command output I have ever seen in my entire Unix life
4,11,Hi guys!\n\nI have found that most of the packages and explanation of FP Growth algorithm in dat...,Python implementation of FP Growth algorithm in data mining


In [23]:
#Drop index column
python = python[['title', 'selftext']]
python.head()

,title,selftext
0,SolarY - A Solar System / Space Science Python Library,"Hey Python-Friends,\n\nIn the last couple of weeks and months I used these tough times and lockd..."
1,Question on assignments in Python3: What is the value of b after changing the value of a ?,"a = 5 \nb = a \na = 11 \nMy question, as a beginner in python, is what is the value of b aft..."
2,CSV - Copy Certain Columns From One CV to Another,Can anyone give me any direction on how to code in Python a script that copies 2 or more columns...
3,This is the most outrageous command output I have ever seen in my entire Unix life,"Today, I just decided to make some Tensorflow stuffs in Python. (I don't use Python for \~3 year..."
4,Python implementation of FP Growth algorithm in data mining,Hi guys!\n\nI have found that most of the packages and explanation of FP Growth algorithm in dat...


In [24]:
#Check Dataset
SQL.head()

,selftext,title
0,Database: Redshift\n\nHi everyone! I have a question. I'm trying to figure out how long a user v...,Find the next timestamp based on where condition per user
1,1) A car salesman at Capital Motors gets a tip that other salesmen are selling cars for way chea...,SQL help please
2,&amp;#x200B;\n\nhttps://preview.redd.it/xh1ted7yeiw51.png?width=1317&amp;format=png&amp;auto=web...,"hi can someone help me with this homework question. i tried B and D, they was wrong"
3,I posted about a similar error yesterday: https://www.reddit.com/r/SQL/comments/jlk44b/programmi...,"pymysql.err.ProgrammingError) (1064, ""You have an error in your SQL syntax; ... again"
4,I was hesitant to call this a many to many relationship but basically I need to import this big ...,Many to Many relationship with single column table


In [25]:
#Reset Index
SQL = SQL.reset_index()
SQL.head()

,index,selftext,title
0,0,Database: Redshift\n\nHi everyone! I have a question. I'm trying to figure out how long a user v...,Find the next timestamp based on where condition per user
1,1,1) A car salesman at Capital Motors gets a tip that other salesmen are selling cars for way chea...,SQL help please
2,2,&amp;#x200B;\n\nhttps://preview.redd.it/xh1ted7yeiw51.png?width=1317&amp;format=png&amp;auto=web...,"hi can someone help me with this homework question. i tried B and D, they was wrong"
3,3,I posted about a similar error yesterday: https://www.reddit.com/r/SQL/comments/jlk44b/programmi...,"pymysql.err.ProgrammingError) (1064, ""You have an error in your SQL syntax; ... again"
4,4,I was hesitant to call this a many to many relationship but basically I need to import this big ...,Many to Many relationship with single column table


In [26]:
#Drop index column
SQL = SQL[['title', 'selftext']]
SQL.head()

,title,selftext
0,Find the next timestamp based on where condition per user,Database: Redshift\n\nHi everyone! I have a question. I'm trying to figure out how long a user v...
1,SQL help please,1) A car salesman at Capital Motors gets a tip that other salesmen are selling cars for way chea...
2,"hi can someone help me with this homework question. i tried B and D, they was wrong",&amp;#x200B;\n\nhttps://preview.redd.it/xh1ted7yeiw51.png?width=1317&amp;format=png&amp;auto=web...
3,"pymysql.err.ProgrammingError) (1064, ""You have an error in your SQL syntax; ... again",I posted about a similar error yesterday: https://www.reddit.com/r/SQL/comments/jlk44b/programmi...
4,Many to Many relationship with single column table,I was hesitant to call this a many to many relationship but basically I need to import this big ...


In [27]:
#Check the shape again
python.shape, SQL.shape

((17868, 2), (11132, 2))

# Tokenizing

In [28]:
python.head(5)

,title,selftext
0,SolarY - A Solar System / Space Science Python Library,"Hey Python-Friends,\n\nIn the last couple of weeks and months I used these tough times and lockd..."
1,Question on assignments in Python3: What is the value of b after changing the value of a ?,"a = 5 \nb = a \na = 11 \nMy question, as a beginner in python, is what is the value of b aft..."
2,CSV - Copy Certain Columns From One CV to Another,Can anyone give me any direction on how to code in Python a script that copies 2 or more columns...
3,This is the most outrageous command output I have ever seen in my entire Unix life,"Today, I just decided to make some Tensorflow stuffs in Python. (I don't use Python for \~3 year..."
4,Python implementation of FP Growth algorithm in data mining,Hi guys!\n\nI have found that most of the packages and explanation of FP Growth algorithm in dat...


In [29]:
#Test on tokenizing selftext
tokenizer = RegexpTokenizer(r'\w+')
tokenizer.tokenize(python['selftext'][0].lower())

['hey',
 'python',
 'friends',
 'in',
 'the',
 'last',
 'couple',
 'of',
 'weeks',
 'and',
 'months',
 'i',
 'used',
 'these',
 'tough',
 'times',
 'and',
 'lockdown',
 'periods',
 'to',
 'write',
 'down',
 'my',
 'python',
 'knowledge',
 'in',
 'a',
 'particular',
 'scientific',
 'field',
 'space',
 'science',
 'more',
 'precisely',
 'space',
 'science',
 'within',
 'the',
 'solar',
 'system',
 'i',
 'studied',
 'physics',
 'later',
 'astro',
 'and',
 'geophysics',
 'and',
 'was',
 'focusing',
 'on',
 'small',
 'bodies',
 'in',
 'our',
 'cosmic',
 'vicinity',
 'namely',
 'asteroids',
 'near',
 'earth',
 'objects',
 'meteors',
 'and',
 'cosmic',
 'dust',
 'during',
 'that',
 'time',
 'and',
 'later',
 'during',
 'my',
 'doctorate',
 'studies',
 'i',
 'worked',
 'on',
 'esa',
 's',
 'comet',
 'mission',
 'rosetta',
 'philae',
 'and',
 'later',
 'i',
 'was',
 'part',
 'of',
 'the',
 'cosmic',
 'dust',
 'analyzer',
 'team',
 'on',
 'board',
 'the',
 'nasa',
 'mission',
 'cassini',
 'huyge

In [30]:
def title_to_words(raw_review):
    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(selftext.lower())    # tokenizing selftext
    stops = set(stopwords.words('english'))                   # setting stop words
    meaningful_words = [w for w in words if not w in stops]   # removing stop words from titles
    return(" ".join(meaningful_words))                        # return the title as a list

In [31]:
import nltk
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

from wordcloud import WordCloud, STOPWORDS

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [32]:
#Create a list for Cleaned Titles for Python
cleaned_selftext_python = []
for selftext in python['selftext']:
    cleaned_selftext_python.append(title_to_words(selftext))

In [33]:
#Put cleaned_titles_python into a dataframe
df_clean_python = pd.DataFrame(cleaned_selftext_python)
df_clean_python.head(2)

,0
0,hey python friends last couple weeks months used tough times lockdown periods write python knowl...
1,5 b 11 question beginner python value b changes value error 5


In [34]:
#Rename column header
df_clean_python.rename(columns={0:'selftext'}, inplace=True)

In [35]:
#Check 
df_clean_python.head()

,selftext
0,hey python friends last couple weeks months used tough times lockdown periods write python knowl...
1,5 b 11 question beginner python value b changes value error 5
2,anyone give direction code python script copies 2 columns csv file neww csv file
3,today decided make tensorflow stuffs python use python 3 years first command python version outp...
4,hi guys found packages explanation fp growth algorithm data mining outdated include much explana...


In [36]:
#Check if there are any empty title
df_clean_python.isna().sum()

selftext    0
dtype: int64

In [37]:
#Add a column to Indicate if it's from Python
df_clean_python['Python'] = 1

In [38]:
#Check if column is successfully added
df_clean_python.head()

,selftext,Python
0,hey python friends last couple weeks months used tough times lockdown periods write python knowl...,1
1,5 b 11 question beginner python value b changes value error 5,1
2,anyone give direction code python script copies 2 columns csv file neww csv file,1
3,today decided make tensorflow stuffs python use python 3 years first command python version outp...,1
4,hi guys found packages explanation fp growth algorithm data mining outdated include much explana...,1


In [39]:
#Create a list for Cleaned Titles for R
cleaned_selftext_SQL = []
for selftext in SQL['selftext']:
    cleaned_selftext_SQL.append(title_to_words(selftext))

In [40]:
#Put cleaned_titles_R into a dataframe
df_clean_SQL = pd.DataFrame(cleaned_selftext_SQL)
df_clean_SQL.head(2)

,0
0,database redshift hi everyone question trying figure long user views certain type page using tim...
1,1 car salesman capital motors gets tip salesmen selling cars way cheaper msrp increase sales rig...


In [41]:
#Rename column header
df_clean_SQL.rename(columns={0:'selftext'}, inplace=True)

In [42]:
#Check
df_clean_SQL.head()

,selftext
0,database redshift hi everyone question trying figure long user views certain type page using tim...
1,1 car salesman capital motors gets tip salesmen selling cars way cheaper msrp increase sales rig...
2,amp x200b https preview redd xh1ted7yeiw51 png width 1317 amp format png amp auto webp amp fdec6...
3,posted similar error yesterday https www reddit com r sql comments jlk44b programmingerror_pymys...
4,hesitant call many many relationship basically need import big dataset car models structure http...


In [43]:
#Add a column to Indicate if it's from Python
df_clean_SQL['Python'] = 0

In [44]:
#Check if column is successfully added
df_clean_SQL.head()

,selftext,Python
0,database redshift hi everyone question trying figure long user views certain type page using tim...,0
1,1 car salesman capital motors gets tip salesmen selling cars way cheaper msrp increase sales rig...,0
2,amp x200b https preview redd xh1ted7yeiw51 png width 1317 amp format png amp auto webp amp fdec6...,0
3,posted similar error yesterday https www reddit com r sql comments jlk44b programmingerror_pymys...,0
4,hesitant call many many relationship basically need import big dataset car models structure http...,0


In [45]:
#Check if any empty
df_clean_SQL.isna().sum()

selftext    0
Python      0
dtype: int64

In [46]:
#Drop data if ""
to_drop_df_clean_python = df_clean_python.loc[df_clean_python['selftext'] == ""].index
df_clean_python.drop(to_drop_df_clean_python , inplace=True)

In [47]:
#Check
df_clean_python.loc[df_clean_python['selftext'] == ""]

,selftext,Python


In [48]:
#Drop data if ""
to_drop_df_clean_SQL = df_clean_SQL.loc[df_clean_SQL['selftext'] == ""].index
df_clean_SQL.drop(to_drop_df_clean_SQL , inplace=True)

In [49]:
df_clean_SQL.loc[df_clean_SQL['selftext'] == ""]

,selftext,Python


In [50]:
#Check shape
df_clean_python.shape, df_clean_SQL.shape

((17857, 2), (11120, 2))

In [51]:
#For both python and R, only select the first 10000
#This will allow us to have equal groups
df_clean_python = df_clean_python.head(5000)
df_clean_SQL = df_clean_SQL.head(5000)

In [52]:
#Check shape
df_clean_python.shape, df_clean_SQL.shape

((5000, 2), (5000, 2))

In [53]:
#Combine Python and R
overall = df_clean_python.append(df_clean_SQL)

In [54]:
#Check head
overall.head()

,selftext,Python
0,hey python friends last couple weeks months used tough times lockdown periods write python knowl...,1
1,5 b 11 question beginner python value b changes value error 5,1
2,anyone give direction code python script copies 2 columns csv file neww csv file,1
3,today decided make tensorflow stuffs python use python 3 years first command python version outp...,1
4,hi guys found packages explanation fp growth algorithm data mining outdated include much explana...,1


In [55]:
#Check tail
overall.tail()

,selftext,Python
5000,technical interview coming looking intermediate sql skills one example provided ctes else lookin...,0
5001,site similar leetcode sql ideally provide test database dump know hackerrank hard sql problems s...,0
5002,may stupid question starting learn sql wondering symbol used looks like used end every line,0
5003,select track dbms_random value lt select 5 count 10 track order dbms_random value rownum lt 5 ge...,0
5004,changing produce different results select title album artist_id select artist_id artist name lik...,0


In [56]:
#Check if empty
overall.isna().sum()

selftext    0
Python      0
dtype: int64

In [57]:
#Check if empty
overall.loc[overall['selftext'] == ""]

,selftext,Python


In [58]:
#Check shape
overall.shape

(10000, 2)

In [59]:
#Save csv
df_clean_python.to_csv('df_clean_python.csv', index=False)
df_clean_SQL.to_csv('df_clean_SQL.csv', index=False)
overall.to_csv('overall.csv', index=False)